In [1]:
import json
import pandas as pd
from dateutil import parser
import time



In [9]:
# Load the JSON data
with open('data/twitter-50mb.json') as f:
    data = json.load(f)
    tweets_df = pd.json_normalize(data['rows'])

    # tweets_df = pd.DataFrame()
    # tweets_df['created_at'] = pd.to_datetime(tweets_df['doc.data.created_at'])
    tweets_df['doc.data.created_at'] = pd.to_datetime(tweets_df['doc.data.created_at'])
    # tweets_df['sentiment_score'] = tweets_df['doc.data.sentiment'] 

    tweets_df['hour'] = tweets_df['doc.data.created_at'].dt.hour
    tweets_df['day'] = tweets_df['doc.data.created_at'].dt.date

    happiest_hour = tweets_df.groupby('hour')['doc.data.sentiment'].sum().idxmax()
    happiest_day = tweets_df.groupby('day')['doc.data.sentiment'].sum().idxmax()
    most_active_hour = tweets_df['hour'].value_counts().idxmax()
    most_active_day = tweets_df['day'].value_counts().idxmax()


    print(f"Happiest hour: {happiest_hour}")
    print(f"Happiest day: {happiest_day}")
    print(f"Most active hour: {most_active_hour}")
    print(f"Most active day: {most_active_day}")



Happiest hour: 2.0
Happiest day: 2021-06-21
Most active hour: 1.0
Most active day: 2021-06-21


In [16]:
df = pd.json_normalize(data['rows'])

tweets_df = pd.DataFrame()
tweets_df['created_at'] = pd.to_datetime(df['doc.data.created_at'])
tweets_df['sentiment_score'] = df['doc.data.sentiment'] 

tweets_df['hour'] = tweets_df['created_at'].dt.hour
tweets_df['day'] = tweets_df['created_at'].dt.date

happiest_hour = tweets_df.groupby('hour')['sentiment_score'].sum().idxmax()
happiest_day = tweets_df.groupby('day')['sentiment_score'].sum().idxmax()
most_active_hour = tweets_df['hour'].value_counts().idxmax()
most_active_day = tweets_df['day'].value_counts().idxmax()


print(f"Happiest hour: {happiest_hour}")
print(f"Happiest day: {happiest_day}")
print(f"Most active hour: {most_active_hour}")
print(f"Most active day: {most_active_day}")

Happiest hour: 2.0
Happiest day: 2021-06-21
Most active hour: 1.0
Most active day: 2021-06-21


In [15]:
tweets = []
for tweet in data['rows']:
    # if 'value' not in tweet:
    #     continue
    if 'doc' not in tweet:
        continue
    if 'sentiment' not in tweet['doc']['data']:
        continue
    # created_at = datetime.fromisoformat(tweet['doc']['data']['created_at'])
    created_at = parser.parse(tweet['doc']['data']['created_at'])
    sentiment_score = tweet['doc']['data']['sentiment']
    if isinstance(sentiment_score, dict):
        sentiment_score = sentiment_score['score']
    # print(sentiment_score)
    tweets.append({'created_at': created_at, 'sentiment_score': sentiment_score})


tweets_df = pd.DataFrame(tweets)
tweets_df['created_at'] = pd.to_datetime(df['doc.data.created_at'])
tweets_df['sentiment_score'] = df['doc.data.sentiment'] 

tweets_df['hour'] = tweets_df['created_at'].dt.hour
tweets_df['day'] = tweets_df['created_at'].dt.date

happiest_hour = tweets_df.groupby('hour')['sentiment_score'].sum().idxmax()
happiest_day = tweets_df.groupby('day')['sentiment_score'].sum().idxmax()
most_active_hour = tweets_df['hour'].value_counts().idxmax()
most_active_day = tweets_df['day'].value_counts().idxmax()


print(f"Happiest hour: {happiest_hour}")
print(f"Happiest day: {happiest_day}")
print(f"Most active hour: {most_active_hour}")
print(f"Most active day: {most_active_day}")

Happiest hour: 1
Happiest day: 2021-06-21
Most active hour: 1
Most active day: 2021-06-21


In [1]:
import dask.dataframe as dd
import pandas as pd

# Read the JSON data in chunks
tweets_ddf = dd.read_json('data/twitter-50mb.json')

# # Process the data in parallel
# tweets_ddf['created_at'] = dd.to_datetime(tweets_ddf['value.doc.data.created_at'])
# tweets_ddf['sentiment_score'] = tweets_ddf['value.doc.data.sentiment']
# tweets_ddf['hour'] = tweets_ddf['created_at'].dt.hour
# tweets_ddf['day'] = tweets_ddf['created_at'].dt.date

# # Calculate the required metrics in parallel
# happiest_hour = tweets_ddf.groupby('hour')['sentiment_score'].mean().nlargest(1).idxmax().compute()
# happiest_day = tweets_ddf.groupby('day')['sentiment_score'].mean().nlargest(1).idxmax().compute()
# most_active_hour = tweets_ddf['hour'].value_counts().idxmax().compute()
# most_active_day = tweets_ddf['day'].value_counts().idxmax().compute()

# print(f"Happiest hour: {happiest_hour}")
# print(f"Happiest day: {happiest_day}")
# print(f"Most active hour: {most_active_hour}")
# print(f"Most active day: {most_active_day}")

ValueError: An error occurred while calling the read_json method registered to the pandas backend.
Original Message: Expected object or value

In [5]:
pd.read_json('data/twitter-50mb.json')

,total_rows,offset,rows
0,185875709,0,"{'id': '1406813874750300166', 'key': [2021, 6,..."
1,185875709,0,"{'id': '1406814402272137216', 'key': [2021, 6,..."
2,185875709,0,"{'id': '1406920780328312832', 'key': [2021, 6,..."
3,185875709,0,"{'id': '1406929052724654084', 'key': [2021, 6,..."
4,185875709,0,"{'id': '1406932228165099530', 'key': [2021, 6,..."
...,...,...,...
49995,185875709,0,"{'id': '1406801815958528009', 'key': [2021, 6,..."
49996,185875709,0,"{'id': '1406800773623672833', 'key': [2021, 6,..."
49997,185875709,0,"{'id': '1406807450959843328', 'key': [2021, 6,..."
49998,185875709,0,"{'id': '1406810512587771905', 'key': [2021, 6,..."


In [2]:
import re

text = '{"text": "@pup_chazable Such a cute pupper 😍",\
          "sentiment": 0.7142857142857143\
        },\
        {"sentiment":-0.09523809523809523, "text": "yes"}'
pattern = r'"sentiment":\s*(-?\d+\.\d+)'

match = re.findall(pattern, text)
print(match)




['0.7142857142857143', '-0.09523809523809523']


In [8]:
DATE_PATTERN = re.compile(r'"created_at":\s*"([^"]+)"')
SENTIMENT_PATTERN = re.compile(r'"sentiment":\s*(-?\d+\.\d+)')

line = '{"text": "@pup_chazable Such a cute pupper 😍",\
          "sentiment": {"score": 0.7142857142857143\
        },\
        {"text": "yes", "created_at": "2013-"}'
        
created_at_match = DATE_PATTERN.search(line)
sentiment_match = SENTIMENT_PATTERN.search(line)

if created_at_match:
  print(created_at_match)
print(sentiment_match)

<re.Match object; span=(131, 152), match='"created_at": "2013-"'>
None
